## Read the data from the database

So it's easier to access the data in case the kernel crashes and had to re-run the codes again

In [ ]:
# Read the table
df = pd.read_sql("SELECT * FROM climate_policy_radar", engine)
df.head()


# 2. Embeddings generation

## 2.1 Load climateBERT

In [ ]:
EMBEDDING_MODEL_LOCAL_DIR = os.getenv('EMBEDDING_MODEL_LOCAL_DIR')
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL")

In [ ]:
# Download
tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL, use_auth_token=False)
model = AutoModelForMaskedLM.from_pretrained(EMBEDDING_MODEL, use_auth_token=False)

# Save it to a  local_models folder
tokenizer.save_pretrained(EMBEDDING_MODEL_LOCAL_DIR)
model.save_pretrained(EMBEDDING_MODEL_LOCAL_DIR)

In [ ]:
# Load the embedding model
tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_LOCAL_DIR)
model = AutoModel.from_pretrained(EMBEDDING_MODEL_LOCAL_DIR)

### Checking existing documents' country


In [ ]:
query = """
SELECT DISTINCT "document_metadata.geographies"
FROM climate_policy_radar
WHERE "document_metadata.geographies" IS NOT NULL;
"""

geos = pd.read_sql(query, engine)
print(geos)


## 2.2 Embedding all documents for all countries

Generate embeddings for all documents and upload them into the database.

A new table is needed, this will be created through the create_table.sql file.So go to create_table.sql and run the query to create the table. Remember to select the Postgres Server at the bottom, and highlight the code and right click to run query. This will create a new table in the database.

In [ ]:
# Make sure column is string for filtering
df["document_metadata.geographies"] = df["document_metadata.geographies"].astype(str)

# Extract 3-letter codes like 'ALB', 'DEU', etc.
df["country_code"] = df["document_metadata.geographies"].str.extract(r"\{(\w+)\}")

# Get all unique codes
country_codes = df["country_code"].dropna().unique()

# Store each country chunk in a dictionary
country_chunks = {}

for code in tqdm(country_codes, desc="Filtering by country"):
    country_chunks[code] = df[df["country_code"] == code]

In [ ]:
print(len(country_chunks))

#how many documents for each country
for code, chunk in country_chunks.items():
    print(f"{code}: {len(chunk)} documents")

In [ ]:
# Ensure the "data" directory exists
os.makedirs("data", exist_ok=True)

# Batch size for embedding
batch_size = 10000

from sqlalchemy.orm import sessionmaker


load_dotenv()
engine = create_engine(os.getenv("DB_URL"))
Session = sessionmaker(bind=engine)
session = Session()

# Loop through each country's data in the dictionary
for code, chunk in tqdm(country_chunks.items(), desc="Processing all countries"):
    original_texts = chunk["text_block.text"]
    doc_ids = chunk["document_id"]
    source_urls = chunk["document_metadata.source_url"]  # ✅ new

    num_batches = (len(texts) + batch_size - 1) // batch_size

    all_embeddings = []
    all_doc_ids = []
    all_texts = []
    all_urls = []

    for i in tqdm(range(num_batches), desc=f"Embedding {code}", leave=False):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(texts))

        batch_texts = texts.iloc[start_idx:end_idx].reset_index(drop=True)
        batch_ids = doc_ids.iloc[start_idx:end_idx].reset_index(drop=True)
        batch_urls = source_urls.iloc[start_idx:end_idx].reset_index(drop=True)

        mask = batch_texts.apply(lambda x: isinstance(x, str) and x.strip() != "")
        batch_texts = batch_texts[mask]
        batch_ids = batch_ids[mask]
        batch_urls = batch_urls[mask]

        batch_embeddings = batch_texts.progress_apply(
            lambda text: generate_embeddings_for_text(text, model, tokenizer)
        )

        all_embeddings.extend(batch_embeddings)
        all_doc_ids.extend(batch_ids)
        all_texts.extend(batch_texts)
        all_urls.extend(batch_urls)

    # Upload to DB
    for doc_id, embedding, original_text, url in tqdm(
        zip(all_doc_ids, all_embeddings, all_texts, all_urls),
        total=len(all_doc_ids),
        desc=f"Uploading {code}"
    ):
        stmt = text("""
            INSERT INTO document_embeddings (document_id, country_code, original_text, source_hyperlink, embedding)
            VALUES (:document_id, :country_code, :original_text, :source_hyperlink, :embedding)
        """)
        session.execute(stmt, {
            "document_id": doc_id,
            "country_code": code,
            "original_text": original_text,
            "source_hyperlink": url,
            "embedding": embedding
        })


    session.commit()


print("✅ All embeddings and original texts uploaded directly.")
